In [ ]:
from pygraphblas import *
from _pygraphblas import lib
import pygraphblas.descriptor
import csv
import sys
import logging
import glob
import operator
from data_loader import DataLoader

In [ ]:
# Setup logger
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter('%(asctime)s %(levelname)-5s %(message)s'))
log = logging.getLogger(__name__)
log.propagate = False
log.addHandler(handler)
log.setLevel(logging.INFO)

In [ ]:
data_dir = 'sf1k-converted/'
loader = DataLoader(data_dir)

vertices, mappings, matrices = loader.load_all_csvs()

In [302]:
def naive_bfs_levels(matrix, source):
    '''
    Input:
        matrix: adjacency matrix describing the graph
        source: source node index
    Returns:
        result_vector: vector of hops to all other nodes
    '''
    
    result_vector = Vector.from_type(UINT64, matrix.nrows)  
    known_nodes_vector = Vector.from_type(BOOL, matrix.nrows)
    
    known_nodes_vector[source] = True
    not_done = True
    level = 1
    
    while not_done and level <= matrix.nrows:
        result_vector[:,known_nodes_vector] = level
        known_nodes_vector = result_vector.vxm(matrix, mask=result_vector, 
                  desc=descriptor.ooco)
        not_done = known_nodes_vector.reduce_bool()
        level += 1
    return result_vector

In [282]:
def MemberFriends(t):
    tagID = mappings['tag'][t]
    tagVector = Vector.from_type(BOOL,matrices['hasTag'].ncols)
    tagVector[tagID] = True
    relevantForumsVector = tagVector.vxm(matrices['hasTag'].transpose())
    relevantPeopleVector = relevantForumsVector.vxm(matrices['hasMember'])
    resultMatrix = Matrix.from_type(BOOL,relevantPeopleVector.nvals,relevantPeopleVector.nvals) 
    matrices['knows'].extract_matrix(relevantPeopleVector.to_lists()[0],relevantPeopleVector.to_lists()[0],out=resultMatrix)
    
    n=0
    idList=[]
    for value in relevantPeopleVector:
        idList+=[[n,value[0]]]
        n += 1

    return resultMatrix,idList

In [307]:
def query4(k,t):
    matrix,idList = MemberFriends(t)
    resultList = []
    for value in idList:
        bfsResultVector = naive_bfs_levels(matrix,value[0])
        n = idsVector.nvals
        r = bfsResultVector.nvals
        s = 0
        for entry in bfsResultVector:
            s += entry[1]-1
        score = ( (r-1)*(r-1) ) / ( (n-1) * s )
        resultList += [[vertices['person'][value[1]],score]]

    resultList.sort(key=operator.itemgetter(0))    
    resultList.sort(key=operator.itemgetter(1), reverse=True)

    for value in resultList[:k]:
        print(value)

In [308]:
#'Bill_Clinton'.id = 2779
query4(3,2779)

[385, 0.5290135396518375]
[492, 0.5259615384615385]
[819, 0.5249520153550864]
